In [1]:
import pandas as pd

1. El “domain_id” no se puede repetir en 4 posiciones consecutivas.
2. El “vertical” no se puede repetir en 1 posición consecutiva.
3. De existir el id 641416750 en el listado debe estar en la posición 3 siendo esta regla
más fuerte que las demás.
4. De existir el id 22351223 en el listado debe estar en la posición 6 siendo esta regla
más fuerte que las demás.
5. Las posiciones 9,10,11 deben tener sí o sí items de la categoría “HOME&DECO”
siendo esta regla más fuerte que la 1 y 2.
6. Cumpliendo estas condiciones, el ordenamiento debe respetar un ordenamiento de
mayor score a menor

In [2]:
df = pd.read_csv('../data/Desafio Ds 2025 - Gen AI/ordenamiento.csv')  # TODO: leer como JSON me hace mas sentido

In [3]:
df

,item_id,vertical,category,domain,score
0,512208310,CPG,PETS FOOD,MLC-CATS_AND_DOGS_FOODS,0.0272
1,468513076,CE,ELECTRONICS,MLC-RANGES,0.9256
2,614337410,CE,ELECTRONICS,MLC-SMART_SPEAKERS,0.8304
3,634351318,APP & SPORTS,APPAREL,MLC-PANTS,0.0560
4,528383704,ACC,VEHICULAR MULTIMEDIA,MLC-GPS,0.2334
...,...,...,...,...,...
4995,621100365,HOME & INDUSTRY,HOME&DECOR,MLC-HOME_LIGHTING_SUPPLIES,0.8520
4996,613555182,BEAUTY & HEALTH,PHARMACEUTICS,MLC-HUMIDIFIERS_AND_VAPORIZERS,0.9298
4997,535350054,HOME & INDUSTRY,STATIONARY,MLC-PENCIL_CASES,0.7832
4998,549193472,HOME & INDUSTRY,HOME&DECOR,MLC-CHRISTMAS_LIGHTS,0.2830
